In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_02a_why_sqrt5")' FastaiNotebook_02a_why_sqrt5

In [ ]:
import FastaiNotebook_02a_why_sqrt5

In [ ]:
// export
import Path
import TensorFlow

### Data

In [ ]:
var (xTrain,yTrain,xValid,yValid) = loadMNIST(path: Path.home/".fastai"/"data"/"mnist_tst", flat: true)

In [ ]:
let trainMean = xTrain.mean()
let trainStd  = xTrain.std()

In [ ]:
xTrain = normalize(xTrain, mean: trainMean, std: trainStd)
xValid = normalize(xValid, mean: trainMean, std: trainStd)

In [ ]:
let (n,m) = (Int(xTrain.shape[0]),Int(xTrain.shape[1]))
let c = yTrain.max()+1
print(n,m,c)

Those can't be used to define a model cause they're not Ints though...

In [ ]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [ ]:
public struct MyModel: Layer {
    public var layer1: Dense<Float>
    public var layer2: Dense<Float>
    
    public init(nIn: Int, nHid: Int, nOut: Int){
        layer1 = Dense(inputSize: nIn, outputSize: nHid, activation: relu)
        layer2 = Dense(inputSize: nHid, outputSize: nOut)
    }
    
    @differentiable
    public func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: layer1, layer2)
    }
}

In [ ]:
var model = MyModel(nIn: m, nHid: nHid, nOut: c)

In [ ]:
let pred = model.applied(to: xTrain)

### Cross entropy loss

Convert this part of 03 to get to...

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let exped = exp(activations) 
    return log(exped / exped.sum(alongAxes: -1))
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
yTrain[0..<3]

In [ ]:
(smPred[0][5],smPred[1][0],smPred[2][4])

There is no fancy indexing yet so we have to use gather to get the indices we want out of our softmaxed predictions.

In [ ]:
func nll<Scalar>(_ input: Tensor<Scalar>, _ target :Tensor<Int32>) -> Tensor<Scalar> 
    where Scalar:TensorFlowFloatingPoint{
        let idx: Tensor<Int32> = Raw.range(start: Tensor(0), limit: Tensor(60000), delta: Tensor(1))
        let indices = Raw.concat(concatDim: Tensor(1), [idx.expandingShape(at: 1), target.expandingShape(at: 1)])
        let losses = Raw.gatherNd(params: input, indices: indices)
        return -losses.mean()
    }

In [ ]:
nll(smPred, yTrain)

In [ ]:
time(repeating: 100){ let _ = nll(smPred, yTrain) }

Simplify `logSoftmax` with log formulas.

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - log(exp(activations).sum(alongAxes: -1))
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
nll(smPred, yTrain)

Use LogSumExp trick

In [ ]:
smPred.max(alongAxes: -1).shape

In [ ]:
func logSumExp<Scalar>(_ x: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let m = x.max(alongAxes: -1)
    return m + log(exp(x-m).sum(alongAxes: -1))
}

In [ ]:
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - logSumExp(activations)
}

In [ ]:
let smPred = logSoftmax(pred)

In [ ]:
nll(smPred, yTrain)

In S4TF nll loss is combined with softmax in:

In [ ]:
let loss = softmaxCrossEntropy(logits: pred, labels: yTrain)
loss

In [ ]:
time(repeating: 100){ let _ = nll(logSoftmax(pred), yTrain)}

In [ ]:
time(repeating: 100){ let _ = softmaxCrossEntropy(logits: pred, labels: yTrain)}

## Basic training loop

Basically the training loop repeats over the following steps:
- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update said parameters with those gradients to make them a little bit better

In [ ]:
// export
public func accuracy(_ output: Tensor<Float>, _ target: Tensor<Int32>) -> Tensor<Float>{
    let corrects = Tensor<Float>(output.argmax(squeezingAxis: 1) .== target)
    return corrects.mean()
}

In [ ]:
print(accuracy(pred, yTrain))

In [ ]:
let bs:Int32=64                         // batch size
let xb = xTrain[0..<bs]          // a mini-batch from x
let preds = model.applied(to: xb) //predictions
print(preds[0], preds.shape)

In [ ]:
let yb = yTrain[0..<bs]
let loss = softmaxCrossEntropy(logits: preds, labels: yb)

In [ ]:
print(accuracy(preds, yb))

In [ ]:
let lr:Float = 0.5   // learning rate
let epochs = 1      // how many epochs to train for

To get all the gradients we need a training context.

In [ ]:
let trainingContext = Context(learningPhase: .training)

Then we can go

In [ ]:
let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
    let preds = model.applied(to: xb, in: trainingContext)
    return softmaxCrossEntropy(logits: preds, labels: yb)
}

Loop by hand

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        model.layer1.weight -= lr * grads.layer1.weight
        model.layer1.bias   -= lr * grads.layer1.bias
        model.layer2.weight -= lr * grads.layer2.weight
        model.layer2.bias   -= lr * grads.layer2.bias
    }
}

Naming all the parameters is a bit boring. We can use `AllDifferentiableVariables` objects to access them all.

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        var parameters = model.allDifferentiableVariables
        for kp in parameters.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self){ 
            parameters[keyPath: kp] -= lr * grads[keyPath:kp]
        }
    }
}

Then we can use a S4TF optimizer to do the step for us.

In [ ]:
let optimizer = SGD<MyModel, Float>(learningRate: lr)

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

## Dataset

We can create a swift `Dataset` from our arrays. It will automatically batch things for us.

In [ ]:
// export
public struct DataBatch<Inputs: Differentiable & TensorGroup, Labels: TensorGroup>: TensorGroup {
    public var xb: Inputs
    public var yb: Labels    
    
    public init(xb: Inputs, yb: Labels){
        self.xb = xb
        self.yb = yb
    }
}

In [ ]:
let train_ds:Dataset<DataBatch> = Dataset(elements:DataBatch(xb:xTrain, yb:yTrain)).batched(Int64(bs))

In [ ]:
for epoch in 1...epochs{
    for batch in train_ds{
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: batch.xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: batch.yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

This `Dataset` can also do the shuffle for us:

In [ ]:
for epoch in 1...epochs{
    for batch in train_ds.shuffled(){
        let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: batch.xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: batch.yb)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grads)
    }
}

### Training loop

Since we can't have a partially differentiable loss function, we have a lossWithGrads parameters.

In [ ]:
func lossWithGrad(_ model: MyModel, _ context: Context, batch:DataBatch<Tensor<Float>, Tensor<Int32>>
                 ) -> (Tensor<Float>, MyModel.AllDifferentiableVariables) {
    return model.valueWithGradient { model -> Tensor<Float> in
        let predictions = model.applied(to: batch.xb, in: context)
        return softmaxCrossEntropy(logits: predictions, labels: batch.yb)
    }
}

In [ ]:
public func train<Opt: Optimizer, Labels:TensorGroup>(
    _ model: inout Opt.Model,
    on dataset: Dataset<DataBatch<Opt.Model.Input, Labels>>,
    using optimizer: inout Opt,
    lossWithGrad: (Opt.Model, Context, DataBatch<Opt.Model.Input, Labels>
                  ) -> (Tensor<Float>, Opt.Model.AllDifferentiableVariables)
) where Opt.Model.Input: TensorGroup,
        Opt.Model.CotangentVector == Opt.Model.AllDifferentiableVariables
{
    let context = Context(learningPhase: .training)
    for batch in dataset {
        let (loss, 𝛁model) = lossWithGrad(model, context, batch)
        optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
    }
}

In [ ]:
var optimizer = SGD<MyModel, Float>(learningRate: lr)

In [ ]:
train(&model, on: train_ds, using: &optimizer, lossWithGrad: lossWithGrad)

### Export

In [ ]:
notebookToScript(fname: (Path.cwd / "03_minibatch_training.ipynb").string)